In [33]:
pip install flask_sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [35]:
#load libraries

from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta

In [39]:
#configuration of the app
app = Flask('COP1_du_13')
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.secret_key = 'supersecretkey'
db = SQLAlchemy(app)

In [41]:
# Baseline for "volunteer=Benevole"
class Benevole(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(100), nullable=False)
    prenom = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    telephone = db.Column(db.String(20), nullable=False)
    instagram = db.Column(db.String(100), unique=True, nullable=True)

In [43]:
# Baseline for the "task= mission"
class Mission(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    titre = db.Column(db.String(100), nullable=False)
    date = db.Column(db.String(20), nullable=False)
    heure = db.Column(db.String(10), nullable=False)
    nb_places = db.Column(db.Integer, nullable=False)

In [45]:
# Baseline of inscriptions (volunteer register for tasks)
class Inscription(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    benevole_id = db.Column(db.Integer, db.ForeignKey('benevole.id'), nullable=False)
    mission_id = db.Column(db.Integer, db.ForeignKey('mission.id'), nullable=False)
    date_inscription = db.Column(db.DateTime, default=datetime.utcnow)

In [47]:
# Route pour afficher les bénévoles et leur engagement
@app.route('/')
def index():
    benevoles = Benevole.query.all()
    return render_template('benevoles.html', benevoles=benevoles)

In [49]:
# Route pour afficher les missions et permettre l'inscription
@app.route('/missions')
def missions():
    missions = Mission.query.all()
    return render_template('missions.html', missions=missions)


In [51]:
# Route pour inscrire un bénévole à une mission
@app.route('/inscrire', methods=['POST'])
def inscrire():
    benevole_id = request.form['benevole_id']
    mission_id = request.form['mission_id']

    mission = Mission.query.get(mission_id)
    if mission and mission.nb_places > 0:
        inscription = Inscription(benevole_id=benevole_id, mission_id=mission_id)
        db.session.add(inscription)
        mission.nb_places -= 1
        db.session.commit()
        flash('✅ Inscription réussie !')
    else:
        flash('❌ Plus de place disponible pour cette mission.')

    return redirect(url_for('missions'))

In [53]:
# Route pour afficher le classement des bénévoles
@app.route('/classement')
def classement():
    classement = db.session.query(Benevole.nom, Benevole.prenom, db.func.count(Inscription.id).label('nb_missions')) \
        .join(Inscription, Benevole.id == Inscription.benevole_id) \
        .group_by(Benevole.id) \
        .order_by(db.func.count(Inscription.id).desc()) \
        .all()
    return render_template('classement.html', classement=classement)

In [22]:
# Fonction pour envoyer une notification aux bénévoles inactifs
def notifier_inactifs():
    seuil = datetime.utcnow() - timedelta(days=90)
    inactifs = db.session.query(Benevole).outerjoin(Inscription, Benevole.id == Inscription.benevole_id) \
        .filter((Inscription.date_inscription == None) | (Inscription.date_inscription < seuil)).all()

    for benevole in inactifs:
        envoyer_email(benevole.email, "🌟 Rappel : Engagez-vous dans une mission !",
                      f"Bonjour {benevole.prenom}, nous avons remarqué que vous n'avez pas participé à une mission récemment. "
                      "Rejoignez une mission dès maintenant !")

In [55]:
# Fonction d'envoi d'email
def envoyer_email(destinataire, sujet, message):
    expediteur = "tonemail@gmail.com"
    mot_de_passe = "motdepasse"
    msg = MIMEText(message)
    msg["Subject"] = sujet
    msg["From"] = expediteur
    msg["To"] = destinataire

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.login(expediteur, mot_de_passe)
        server.sendmail(expediteur, destinataire, msg.as_string())
        server.quit()
        print(f"📧 Email envoyé à {destinataire}")
    except Exception as e:
        print(f"⚠️ Erreur lors de l'envoi de l'email : {e}")

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(debug=True)


 * Serving Flask app 'COP1_du_13'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
